# Схемы разделения секрета

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime

# Обычный рандом
import random

# Утилитки
from functools import reduce

----

## Схема Шамира
![Shamir scheme](./images/Shamir_1_5.png)
![Shamir scheme](./images/Shamir_2_5.png)

In [4]:
class ShamirParticipant(object):
    def __init__(self, x, y, t):
        """
        Инициализация участника разделения секрета.
        Сохраняются х, у и t.
        """
        
        self._x = x
        self._y = y
        self._t = t
        
        trace('[ShamirParticipant]', f'{x = }, {y = }')
        
    def __eq__(self, other):
        """
        Сравнение на равенство. Сравниваю по х - достаточно.
        """
        
        return self.x == other.x
        
    @property
    def x(self):
        """
        Получение х.
        """
        
        return self._x
    
    
    @property
    def y(self):
        """
        Получение у.
        """
        
        return self._y
        
        
    @property
    def equation(self):
        """
        Получение соответствующего участнику уравнения.
        """
        
        return [1] + [self._x ^ i for i in range(1, self._t)], self._y

In [5]:
class ShamirDealer(object):
    def __init__(self, t, n):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        self._field = GF(getStrongPrime(512))
        self._t = t
        self._n = n
        self._key = self._field.random_element()
        self._coefs = []
        self._values = []
        
        trace('[ShamirDealer]', f't = {self._t}, n = {self._n}')
        trace('[ShamirDealer]', self._field)
        trace('[ShamirDealer]', f'key = {self._key}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._coefs  = [self._field.random_element() for _ in range(self._t - 1)]
        
        self._values = [self._key + sum([self._coefs[i] * x ^ (i + 1) for i in range(self._t - 1)]) 
                        for x in range(1, self._n + 1)]
        
        return [ShamirParticipant(self._field.gen() + i, self._values[i], self._t) 
                for i in range(self._n)]
    
    
    @property
    def field(self):
        """
        Получение поля.
        """
        
        return self._field
    
    
    @property
    def key(self):
        """
        Получение ключа для проверки
        """
        
        return self._key

In [6]:
def restore_secret_by_equation_system(field, participants: list[ShamirParticipant]):
    """
    Восстановление секрета путем решения системы уравнений.
    """
    
    equations = [p.equation for p in participants]
    M = Matrix(field, [eq[0] for eq in equations])
    v = vector(field, (eq[1] for eq in equations))
    
    return M.solve_right(v)[0]  # Нас интересует только первая переменная,
                                # так как именно она является ключом

In [7]:
def restore_secret_by_lagrange_interpolation(participants: list[ShamirParticipant]):
    """
    """
    
    def prod(lst):
        return reduce((lambda x, y: x * y), lst)
    
    return sum([p1.y * prod([p2.x * (p2.x - p1.x).inverse_of_unit() 
                             for p2 in participants if p1 != p2])
                for p1 in participants])

----

In [8]:
t, n = 3, 5

In [9]:
d = ShamirDealer(t, n)

[TRACE] [ShamirDealer] t = 3, n = 5
[TRACE] [ShamirDealer] Finite Field of size 10087599926432669877255757131340494585861344245323103570604870711662061441976589141100941371400637816665472383913879707653154025440571482937622660714432863
[TRACE] [ShamirDealer] key = 9746591256562501218141980966268154993154033221801124817693174139510398087491312698483558949895138502935651877300377630333523291287941623872314496441295855


In [10]:
participants = d.split_key()

[TRACE] [ShamirParticipant] x = 1, y = 7566329960216023655489290763983585687009801059677459891296312764692650642795591017344708559704431241637319884682430574896266530897154785046306611525326316
[TRACE] [ShamirParticipant] x = 2, y = 7808791295896368094401966666937784746745954062260264050182476007217046822529626069553704126741025845694103836153701169552678519261924127923968867510603089
[TRACE] [ShamirParticipant] x = 3, y = 386375337170864657624251543790257586501147984226433723746793155421525184716828714009604279604284498440531347800309706649605230941678169567678603682693311
[TRACE] [ShamirParticipant] x = 4, y = 5474281936904853099667659657221993377998071316222176053199005632630208613310377232914291761095482833207547187450015601493354716817559875852681141470462708
[TRACE] [ShamirParticipant] x = 5, y = 2897311242232993666020676744552002949514035567601283897329372015518974224357093344065883828413345216664206587275059438777618926008426280903731159445045554


In [11]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 9746591256562501218141980966268154993154033221801124817693174139510398087491312698483558949895138502935651877300377630333523291287941623872314496441295855
Restored correctly: True


In [12]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 4901145992508857215011248755790618261393262892388186647127124904826110838631799231787867035440534772225419989121942330146185793776829260464974214638803231
Restored correctly: False


In [13]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 9746591256562501218141980966268154993154033221801124817693174139510398087491312698483558949895138502935651877300377630333523291287941623872314496441295855
Restored correctly: True


In [14]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 6112507308522268215793931808410002444333455474741421189768637213497182650846677598461790014054185704902977961166551155193020168154607351316809285089426387
Restored correctly: False


---
## 